In [1]:
import os
os.chdir('/Users/ltran/Documents/TrueData29/CPC_ML_tutorial/')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import random

# Set data directory
data_dir = '/Users/ltran/Documents/Data/'

# Set working directory
root_dir = '/Users/ltran/Documents/TrueData0104/CPC_ML_tutorial/'
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

isDir = os.path.isdir('/Users/ltran/Documents/Data/Outputs/')
if isDir:
    ! rm -r '/Users/ltran/Documents/Data/Outputs/'
! mkdir '/Users/ltran/Documents/Data/Outputs/'

## Load infos

In [2]:
# Demographic infos
part_infos = pd.read_csv(os.path.join(data_dir, 'participants.tsv'), sep = "\t")
part_infos.ASD = part_infos.ASD.replace('?', np.nan).dropna()

In [3]:
# Scanner infos
instru = pd.read_csv(os.path.join(data_dir, 'TSA_cohort_dcm_info_anonymized.csv'), sep = '\t')

In [4]:
# Merge demo and scanner
infos = part_infos.merge(instru.rename(columns = {'SubCode BIDS' : 'participant_id'}))
infos = infos.loc[infos['magnetic_field_strength'] == '1_5']

In [5]:
# Load volume measures

vol_RDB = pd.read_csv(os.path.join(data_dir,'fs-7.2.0-tfe.aseg.csv'), sep = '\t')

QC

In [23]:
QC_trio = pd.read_csv(os.path.join(data_dir, 'QC_trio.csv'))

In [24]:
QC_trio = QC_trio.dropna(subset='QC')

In [25]:
l_dup = QC_trio[QC_trio.duplicated(subset = 'BIDS')].BIDS

In [ ]:
QC_trio 

In [42]:
set(QC_trio['Protocole'])

{'ISO', 'TFE', 'isotrope', 'tfe'}

In [28]:
QC_trio['Protocole'] = QC_trio['Protocole'].replace({'isotrope': 'ISO', ''})

1       sub-0277
3       sub-0278
5       sub-0279
7       sub-0280
9       sub-0281
          ...   
1721    Sub-0944
1723    Sub-0945
1725    Sub-0946
1727    Sub-0947
1735    Sub-0951
Name: BIDS, Length: 333, dtype: object

In [36]:
for i in l_dup:
    if QC_trio[QC_trio.BIDS == i]['best Vbrain'].isna().all():
        temp = QC_trio[QC_trio.BIDS == i]
    else: 
        print(i, 'not NA')

sub-0277 NA
sub-0278 NA
sub-0279 NA
sub-0280 NA
sub-0281 NA
sub-0282 NA
sub-0283 NA
sub-0284 NA
sub-0285 NA
sub-0286 NA
sub-0287 NA
sub-0288 NA
sub-0289 NA
sub-0290 NA
sub-0291 NA
sub-0292 NA
sub-0293 NA
sub-0298 NA
sub-0299 NA
sub-0300 NA
sub-0301 NA
sub-0302 NA
sub-0303 NA
sub-0304 NA
sub-0306 NA
sub-0307 NA
sub-0308 NA
sub-0309 NA
sub-0310 NA
sub-0311 not NA
sub-0312 not NA
sub-0313 NA
sub-0314 NA
sub-0354 NA
sub-0355 NA
sub-0356 NA
sub-0357 NA
sub-0358 NA
sub-0359 NA
sub-0360 NA
sub-0364 NA
sub-0365 NA
sub-0366 NA
sub-0368 NA
sub-0369 not NA
sub-0370 NA
sub-0373 not NA
sub-0373 not NA
sub-0375 NA
sub-0376 NA
sub-0377 NA
sub-0378 NA
sub-0379 NA
sub-0380 NA
sub-0381 NA
sub-0382 NA
sub-0383 NA
sub-0384 NA
sub-0385 NA
sub-0386 NA
sub-0387 NA
sub-0388 NA
sub-0389 NA
sub-0390 NA
sub-0391 NA
sub-0392 NA
sub-0393 NA
sub-0394 NA
sub-0395 NA
sub-0396 NA
sub-0397 NA
sub-0398 NA
sub-0399 NA
sub-0400 NA
sub-0401 NA
sub-0402 NA
sub-0403 NA
sub-0404 NA
sub-0405 NA
sub-0407 NA
sub-0408 NA
sub-0409

In [41]:
QC_trio[(QC_trio.BIDS == 'sub-0278') & (QC_trio.Protocole == 'TFE')]

,Unnamed: 0,BIDS,session,Protocole,cropped FOV,cropped cerebellum,cropped orbitofrontal,defacing failed,motion,low Quality brain,QC,best Vbrain,Best Vcereb,comment,Unnamed: 13,nombre fail,nombre m,nombre pass,nombre NA
3,3,sub-0278,1.0,TFE,0.0,0.0,0.0,0.0,0.0,1.0,P,NaN,NaN,M1,NaN,NaN,NaN,NaN,NaN


In [27]:

QC_trio[(QC_trio.BIDS.isin(l_dup))]

,Unnamed: 0,BIDS,session,Protocole,cropped FOV,cropped cerebellum,cropped orbitofrontal,defacing failed,motion,low Quality brain,QC,best Vbrain,Best Vcereb,comment,Unnamed: 13,nombre fail,nombre m,nombre pass,nombre NA
0,0,sub-0277,1.0,ISO,0.0,0.0,0.0,0.0,0.0,1.0,P,NaN,NaN,M1,NaN,NaN,NaN,NaN,NaN
1,1,sub-0277,1.0,TFE,0.0,0.0,0.0,0.0,0.0,1.0,P,NaN,NaN,M1,NaN,NaN,NaN,NaN,NaN
2,2,sub-0278,1.0,ISO,0.0,0.0,0.0,0.0,0.0,1.0,P,NaN,NaN,M1,NaN,NaN,NaN,NaN,NaN
3,3,sub-0278,1.0,TFE,0.0,0.0,0.0,0.0,0.0,1.0,P,NaN,NaN,M1,NaN,NaN,NaN,NaN,NaN
4,4,sub-0279,1.0,ISO,0.0,0.0,0.0,0.0,0.0,1.0,P,NaN,NaN,M1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1725,103,Sub-0946,1.0,tfe,0.0,0.0,0.0,0.0,0.0,0.0,P,1.0,1.0,large ventricules,NaN,NaN,NaN,NaN,NaN
1726,104,Sub-0947,1.0,isotrope,0.0,0.0,0.0,0.0,0.0,0.0,P,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1727,105,Sub-0947,1.0,tfe,0.0,0.0,0.0,0.0,0.0,0.0,P,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1734,112,Sub-0951,1.0,tfe,0.0,0.0,0.0,0.0,0.0,0.0,P,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
QC_trio.QC.isna().sum()

1374

In [119]:
l_ses02 = []
for i in vol_RDB['Measure:volume']:
    if 'ses-02' in i:
        l_ses02.append(i)


vol_RDB = vol_RDB.loc[~(vol_RDB['Measure:volume'].isin(l_ses02))]

In [120]:
# Format participant IDs

participant_id = []
for i in vol_RDB['Measure:volume']:
    participant_id.append(int(str(i).split('_')[0].split('-')[1].lstrip('0')))
vol_RDB['participant_id'] = participant_id

In [121]:
# Drop duplicates 
vol_RDB = vol_RDB.drop_duplicates(subset='participant_id')

In [122]:
# Merge volumes with participant infos

vol_RDB = vol_RDB.merge(infos, on = 'participant_id')

In [123]:
# Format sex

vol_RDB.sex = vol_RDB.sex.replace({'F' : -1, 'M' : 1})

In [124]:
# Drop NA

vol_RDB = vol_RDB.dropna(subset = ['sex', 'age_at_first_scan', 'ASD', 'EstimatedTotalIntraCranialVol'])

In [125]:
vol_RDB.loc[(vol_RDB['manufacturer_model_name'] == 'INTERA'), 'site'] = 'RDB_INTERA'
vol_RDB.loc[(vol_RDB['manufacturer_model_name'] == 'INTERA'), 't1_site'] = 9

In [126]:
vol_RDB.loc[(vol_RDB['manufacturer_model_name'] == 'INGENIA'), 'site'] = 'RDB_INGENIA'
vol_RDB.loc[(vol_RDB['manufacturer_model_name'] == 'INGENIA'), 't1_site'] = 10

## Export 

In [127]:
vol_RDB.to_csv(os.path.join(data_dir, 'Outputs/vol_RDB.csv'), index = False)